In [1]:
def setup(): 
    #installing requirements :
    !pip install click
    !pip install networkx
    !pip install nltk
    !pip install numpy
    !pip install editdistance==0.4 #<- only works on Python 3.10!

    !pip install git+https://github.com/davidadamojr/TextRank.git
    !pip install pytextrank
    !textrank initialize
    !python -m spacy download en_core_web_sm
    return

setup() #Does not work with Python 3.11

import pandas as pd
import json
import pytextrank
import spacy

# Load a spacy model
nlp = spacy.load("en_core_web_sm")
# Add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

Das System kann die angegebene Datei nicht finden.


  Cloning https://github.com/davidadamojr/TextRank.git to c:\users\pbrun\appdata\local\temp\pip-req-build-fz53rtnz
  Resolved https://github.com/davidadamojr/TextRank.git to commit f57ed2a3a7cc1c4883f7f026b8d4fa24993f62f7
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/davidadamojr/TextRank.git 'C:\Users\pbrun\AppData\Local\Temp\pip-req-build-fz53rtnz'


Traceback (most recent call last):
  File "C:\Users\pbrun\anaconda3\envs\GLMPascal\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\pbrun\anaconda3\envs\GLMPascal\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\pbrun\anaconda3\envs\GLMPascal\Scripts\textrank.exe\__main__.py", line 4, in <module>
ImportError: cannot import name 'cli' from 'main' (C:\Users\pbrun\switchdrive\GLMPascal\main.py)


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB 1.3 MB/s eta 0:00:10
     --------------------------------------- 0.1/12.8 MB 991.0 kB/s eta 0:00:13
     ---------------------------------------- 0.2/12.8 MB 1.3 MB/s eta 0:00:10
      --------------------------------------- 0.3/12.8 MB 1.6 MB/s eta 0:00:08
      --------------------------------------- 0.3/12.8 MB 1.6 MB/s eta 0:00:08
      --------------------------------------- 0.3/12.8 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.4/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/12.8 MB 

Documentation Textrank: <https://github.com/davidadamojr/TextRank/blob/master/README.md>
Documentation PyTextRank: <https://derwen.ai/docs/ptr/>

# Load & Clean Data

Sources: 
Cleantech: <https://www.kaggle.com/datasets/jannalipenkova/cleantech-media-dataset>
Google patents: <https://www.kaggle.com/datasets/prakharbhandari20/cleantech-google-patent-dataset>
Cleaning process as in phase 1.

In [4]:
# load cleantech data
path="C:/Users/pbrun/Downloads/cleantech_media_dataset_v2_2024-02-23.csv"

df_cleantech_media = pd.read_csv(path, index_col=0, parse_dates=['date'])

# clean up unnecessary symbols
df_cleantech_media.content = df_cleantech_media['content'].str.strip('[]"\'• ')

# drop the five duplicates
df_cleantech_media.drop_duplicates(subset=['content'], inplace=True)
df_cleantech_media

,title,date,author,content,domain,url
1280,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,NaN,Qatar Petroleum ( QP) is targeting aggressive ...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
1281,India Launches Its First 700 MW PHWR,2021-01-15,NaN,Nuclear Power Corp. of India Ltd. ( NPCIL) syn...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
1283,New Chapter for US-China Energy Trade,2021-01-20,NaN,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
1284,Japan: Slow Restarts Cast Doubt on 2030 Energy...,2021-01-22,NaN,The slow pace of Japanese reactor restarts con...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
1285,NYC Pension Funds to Divest Fossil Fuel Shares,2021-01-25,NaN,Two of New York City's largest pension funds s...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...
...,...,...,...,...,...,...
81812,Strata Clean Energy Nets $ 300 Million in Fund...,2023-11-06,NaN,Strata Clean Energy has closed a $ 300 million...,solarindustrymag,https://solarindustrymag.com/strata-clean-ener...
81813,Orsted Deploying SparkCognition Renewable Suit...,2023-11-07,NaN,Global renewable energy developer Ørsted is de...,solarindustrymag,https://solarindustrymag.com/orsted-deploying-...
81814,Veolia Has Plans for 5 MW of Solar in Arkansas,2023-11-07,NaN,"Veolia North America, a provider of environmen...",solarindustrymag,https://solarindustrymag.com/veolia-has-plans-...
81815,"SunEdison: Too Big, Too Fast?",2023-11-08,NaN,Once the self-proclaimed “ leading renewable p...,solarindustrymag,http://www.solarindustrymag.com/online/issues/...


In [19]:
# load google patents data
filepath="C:/Users/pbrun/Downloads/bq-results-20240124-055833-1706076079048.json"

def filter_english(texts):
    for item in texts:
        if item["language"] == "en":
            return item["text"]
    return None  # Return None if no English text is found
def loadAndClean(filepath):
    df = pd.read_json(filepath, lines=True)
    df['title'] = df['title_localized'].apply(filter_english)
    df['abstract'] = df['abstract_localized'].apply(filter_english)
    df['abstract_lang'] = df['abstract_localized'].apply(lambda x: [item['language'] for item in x])
    df = df.dropna(subset=['abstract'])
    df.cpc.values[:5]
    df.drop(columns=['cpc', 'abstract_localized', 'title_localized'], inplace=True)
    df.publication_date = pd.to_datetime(df_google_patents.publication_date, format="%Y%m%d")
    df[df.duplicated(subset=['abstract'], keep=False)]
    df.drop_duplicates(subset=['abstract'], inplace=True)
    return df

df_google_patents=loadAndClean(filepath)
df_google_patents

,publication_number,application_number,country_code,publication_date,inventor,title,abstract,abstract_lang
0,US-2022239235-A1,US-202217717397-A,US,2022-07-28,[],Adaptable DC-AC Inverter Drive System and Oper...,Disclosed is an adaptable DC-AC inverter syste...,[en]
1,US-2022239251-A1,US-202217580956-A,US,2022-07-28,[],System for providing the energy from a single ...,"In accordance with an example embodiment, a so...",[en]
4,US-11396827-B2,US-202117606042-A,US,2022-07-26,[],Control method for optimizing solar-to-power e...,A control method for optimizing a solar-to-pow...,[en]
5,CN-114772674-A,CN-202210500131-A,CN,2022-07-22,[],Low-carbon running saline wastewater treatment...,The invention discloses a system and a method ...,"[zh, en]"
6,CN-217026795-U,CN-202220668705-U,CN,2022-07-22,[],Water ecological remediation device convenient...,The utility model discloses a water ecological...,"[zh, en]"
...,...,...,...,...,...,...,...,...
29993,CN-214850044-U,CN-202120980740-U,CN,2021-11-23,"[WANG HAI, WANG YANG]",Box transformer substation capable of adjustin...,The utility model relates to a case becomes te...,"[en, zh]"
29994,CN-214840532-U,CN-202121225269-U,CN,2021-11-23,"[Gao Zhelu, Dai Jiangyue, LIU ZIYI, YAN CONGQI...",Garden waste anaerobic fermentation coupling m...,The utility model discloses a gardens discarde...,"[en, zh]"
29995,CN-214835218-U,CN-202121309411-U,CN,2021-11-23,"[ZHANG ANZHI, ZHANG WANYONG, HUANG HUIYONG, XU...",Unmanned aerial vehicle hangar with function o...,The utility model relates to an unmanned air v...,"[en, zh]"
29996,CN-113690940-A,CN-202111107806-A,CN,2021-11-23,"[ZHANG YUCHUAN, LONG HAIYANG, TAN CUNZHEN]",Wind power supply control method,The invention provides a wind power supply con...,"[en, zh]"


# Summarize Content with textrank

In [ ]:
def summarizeText(text):
    doc = nlp(text)
    summary = "\n".join([sent.text for sent in doc._.textrank.summary(limit_phrases=15, limit_sentences=3)])
    return summary

In [5]:
# Summarize Cleantech
df_cleantech_media['content_summarized']=df_cleantech_media['content'].apply(summarizeText)
df_cleantech_media

,title,date,author,content,domain,url,summarized
1280,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,NaN,Qatar Petroleum ( QP) is targeting aggressive ...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Qatar Petroleum ( QP) is targeting aggressive ...
1281,India Launches Its First 700 MW PHWR,2021-01-15,NaN,Nuclear Power Corp. of India Ltd. ( NPCIL) syn...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Nuclear Power Corp. of India Ltd. ( NPCIL) syn...
1283,New Chapter for US-China Energy Trade,2021-01-20,NaN,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Trump demanded bigger purchases of US energy p...
1284,Japan: Slow Restarts Cast Doubt on 2030 Energy...,2021-01-22,NaN,The slow pace of Japanese reactor restarts con...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,The NRA has yet to approve applications to cha...
1285,NYC Pension Funds to Divest Fossil Fuel Shares,2021-01-25,NaN,Two of New York City's largest pension funds s...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,The announcement by the New York City pension ...
...,...,...,...,...,...,...,...
81812,Strata Clean Energy Nets $ 300 Million in Fund...,2023-11-06,NaN,Strata Clean Energy has closed a $ 300 million...,solarindustrymag,https://solarindustrymag.com/strata-clean-ener...,"', 'Nomura Securities International Inc. led t..."
81813,Orsted Deploying SparkCognition Renewable Suit...,2023-11-07,NaN,Global renewable energy developer Ørsted is de...,solarindustrymag,https://solarindustrymag.com/orsted-deploying-...,Global renewable energy developer Ørsted is de...
81814,Veolia Has Plans for 5 MW of Solar in Arkansas,2023-11-07,NaN,"Veolia North America, a provider of environmen...",solarindustrymag,https://solarindustrymag.com/veolia-has-plans-...,"Veolia North America, a provider of environmen..."
81815,"SunEdison: Too Big, Too Fast?",2023-11-08,NaN,Once the self-proclaimed “ leading renewable p...,solarindustrymag,http://www.solarindustrymag.com/online/issues/...,Once the self-proclaimed “ leading renewable p...


In [20]:
# Summarize Google Patent data
df_google_patents['abstract_summarized']=df_google_patents['abstract'].apply(summarizeText)
df_google_patents

,publication_number,application_number,country_code,publication_date,inventor,title,abstract,abstract_lang,summarized
0,US-2022239235-A1,US-202217717397-A,US,2022-07-28,[],Adaptable DC-AC Inverter Drive System and Oper...,Disclosed is an adaptable DC-AC inverter syste...,[en],The system includes multiple DC input sources ...
1,US-2022239251-A1,US-202217580956-A,US,2022-07-28,[],System for providing the energy from a single ...,"In accordance with an example embodiment, a so...",[en],"In accordance with an example embodiment, a so..."
4,US-11396827-B2,US-202117606042-A,US,2022-07-26,[],Control method for optimizing solar-to-power e...,A control method for optimizing a solar-to-pow...,[en],Through reading the relevant information of th...
5,CN-114772674-A,CN-202210500131-A,CN,2022-07-22,[],Low-carbon running saline wastewater treatment...,The invention discloses a system and a method ...,"[zh, en]",The salt-containing wastewater is preheated by...
6,CN-217026795-U,CN-202220668705-U,CN,2022-07-22,[],Water ecological remediation device convenient...,The utility model discloses a water ecological...,"[zh, en]","According to the utility model, the fixed slee..."
...,...,...,...,...,...,...,...,...,...
29993,CN-214850044-U,CN-202120980740-U,CN,2021-11-23,"[WANG HAI, WANG YANG]",Box transformer substation capable of adjustin...,The utility model relates to a case becomes te...,"[en, zh]",This case of adjustable solar panel orientatio...
29994,CN-214840532-U,CN-202121225269-U,CN,2021-11-23,"[Gao Zhelu, Dai Jiangyue, LIU ZIYI, YAN CONGQI...",Garden waste anaerobic fermentation coupling m...,The utility model discloses a gardens discarde...,"[en, zh]",The utility model discloses a gardens discarde...
29995,CN-214835218-U,CN-202121309411-U,CN,2021-11-23,"[ZHANG ANZHI, ZHANG WANYONG, HUANG HUIYONG, XU...",Unmanned aerial vehicle hangar with function o...,The utility model relates to an unmanned air v...,"[en, zh]",This unmanned aerial vehicle hangar with funct...
29996,CN-113690940-A,CN-202111107806-A,CN,2021-11-23,"[ZHANG YUCHUAN, LONG HAIYANG, TAN CUNZHEN]",Wind power supply control method,The invention provides a wind power supply con...,"[en, zh]",The invention provides a wind power supply con...
